In [1]:
# Step 1: Accept search term from user and
# download last 100 tweets about it

In [1]:
import twitter

api = twitter.Api(
    consumer_key = "ZrRO6Q1rjB8pjrfT6vS33em1Z",
    consumer_secret = "r2rkhELbheAk4YzgKrZT780KyJnkUhBbMP3x6Hz1DV0qSHGHOS",
    access_token_key = "559857675-V3d6JUsBoMSlXBEZnO37x0bCJHVogMhRTNrl1PAu",
    access_token_secret = "6W3atnDdn3i4IrlddjiEu8XAdfAwUApWXVsqu7gzZUfL3"
)

In [2]:
# Setup function which will accept a search term
# and fetch tweets for that term.

def createTestData(searchString):
    try:
        tweets_fetched = api.GetSearch(searchString, count=100)
        # This returns a list of twitter.Status objects
        # These have attributes for text, hashtags, etc
        return [{"text": status.text, "label": None} for status in tweets_fetched]
    except:
        print "Sorry, there was an error."
        return None
    
search_string = raw_input("What are we searching for today?")
testData = createTestData(search_string)

What are we searching for today?apple


In [4]:
# Step 2: Classify each tweet as positive or negative

# 2a: Download a corpus of tweets which can be used as training data
# We'll use Niek Sander's Tweet Sentiment Corpus (5k+ labelled tweets)
# Write a function to read csv, fetch tweet id and label, then download
# the respective tweet from twitter, then write it back to another csv

# Note: Since twitter has a limit of downloading only 180 tweets per 
# 15 mins, we need around 10 hours for this function to finish downloading
# all the 5000 tweets

def createTrainingCorpus(corpusFile, tweetDataFile):
    import csv
    corpus = []
    with open(corpusFile, "rb") as csvFile:
        lineHeader = csv.reader(csvFile, delimiter=',', quotechar='\"')
        for row in lineHeader:
            corpus.append({"tweet_id": row[2], "label": row[1], "topic": row[0]})
    
    # Twitter has a rate limit of 180 tweets/15 mins. So, we need to use 
    # a delay after each fetch.
    
    import time
    rate_limit = 180
    sleep_time = 900 / 180
    
    trainingData = []
    for tweet in corpus:
        try:
            status = api.GetStatus(tweet['tweet_id'])
            tweet['text'] = status.text
            trainingData.append(tweet)
            try:
                with open(tweetDataFile, 'ab') as csvFile:
                    linewriter = csv.writer(csvFile, delimiter=',', quotechar="\"")
                    linewriter.writerow([tweet['tweet_id'], tweet['text'], tweet['label'], tweet['topic']])
            except:
                continue
            time.sleep(sleep_time)
        except:
            continue
    
    return trainingData

def createTrainingCorpusFromFile(tweetDataFile):
    import csv
    trainingData = []
    with open(tweetDataFile, "rb") as csvFile:
        lineHeader = csv.reader(csvFile, delimiter=',', quotechar='\"')
        for row in lineHeader:
            trainingData.append({
                "tweet_id": row[0],
                "label": row[2],
                "topic": row[3],
                "text": row[1],
            })
    return trainingData
            

corpusFile = "/home/scarecrow/Downloads/sanders-twitter-0.2/corpus.csv"
tweetDataFile = "modifiedCorpus.csv"

# Use this if running for the first time:

trainingData = createTrainingCorpus(corpusFile, tweetDataFile)

# Use this if tweets have been downloaded already and saved to a file
# to save time on downloading tweets again: 
# trainingData = createTrainingCorpusFromFile(tweetDataFile)

In [6]:
#2b: A class to prepare all the tweets. Both test and training

import re
from nltk.tokenize import word_tokenize
from string import punctuation
from nltk.corpus import stopwords

class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER', 'URL', '\'s'])
        
    def processTweets(self, list_of_tweets):
        # List of tweets is a list of dicts
        processedTweets = []
        # List of tuples which is a list of words and its label
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet['text']), tweet['label']))
        return processedTweets
    
    def _processTweet(self, tweet):
        # 1. Convert to lowercase
        tweet = tweet.lower()
        # 2. Replace links with URL
        tweet = re.sub("((www\.[^\s]+)|(https?://[^\s]+))", "URL", tweet)
        # 3. Replace @<username> with AT_USER
        tweet = re.sub("@[^\s]+", "AT_USER", tweet)
        # 4. Replace #word with word
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
        
        tweet = word_tokenize(tweet)
        
        return [word for word in tweet if word not in self._stopwords]
    
tweetProcessor = PreProcessTweets()
ppTrainingData = tweetProcessor.processTweets(trainingData)
ppTestData = tweetProcessor.processTweets(testData)

In [27]:
#2c: Extract features and train your classifier
# Use 2 methods - Naive Baye's and SVM

import nltk

# i) Naive Baye's - We'll use NLTK's built in classifier

# First, build vocabulary
def buildVocabulary(ppTrainingData):
    all_words = []
    for [words, sentiment] in ppTrainingData:
        if sentiment in ['positive', 'negative']:
            all_words.extend(words)
        # words are not deduped
    wordlist = nltk.FreqDist(all_words)
    # This creates a dict of words and its frequencies
    word_features =wordlist.keys() 
    return word_features

# NLTK has a apply_features function, which takes in a user defined function to extract features
# from training data. We want to define our own extract features function to take each tweet in
# the training data and represent it with the presence or absence of a word in the vocabulary.

def extract_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

word_features = buildVocabulary(ppTrainingData)
trainingFeatures = nltk.classify.apply_features(extract_features, [tweet for tweet in ppTrainingData if tweet[1] in ['positive', 'negative']])
NBayesClassifier = nltk.NaiveBayesClassifier.train(trainingFeatures)

In [14]:
# ii) SVM

from nltk.corpus import sentiwordnet as swn
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

# We have to change the format of the data slightly as
# SKLearn has a CountVectorizer object. It will take
# documents and directly return a term-document matrix
# with the frequencies of a word in the document. It 
# builds the vocabulary by itself. We will give the 
# trainingData and the labels separately to the SVM
# classifier and not as tuples.

svmTrainingData = [' '.join(tweet[0]) for tweet in ppTrainingData]
vectorizer = CountVectorizer(min_df=1)
X = vectorizer.fit_transform(svmTrainingData).toarray()
vocabulary = vectorizer.get_feature_names()

# Now, we'll use sentiwordnet to add some weight to these features

swn_weights = []

for word in vocabulary:
    try:
        synset = list(swn.senti_synsets(word))
        
        # use the first synset only to compute the score, as this 
        # is the most common usage of the word.
        
        common_meaning = synset[0]
        
        # If posScore is greater, use that as weight, if neg_score is greater, 
        # use -neg_score as weight.
        
        if common_meaning.pos_score() > common_meaning.neg_score():
            weight = common_meaning.pos_score()
        elif common_meaning.pos_score() < common_meaning.neg_score():
            weight = -1 * common_meaning.neg_score()
        else:
            weight = 0
            
    except Exception as e:
        weight = 0
    
    swn_weights.append(weight)
        
# Let's now multiply each array in our original; matrix with these weights
# Initialize a list

swn_X = []
for row in X:
    swn_X.append(np.multiply(row, np.array(swn_weights)))

# Convert list to np array
swn_X = np.vstack(swn_X[:877])
# Used hack to match shape of swn_X and y for the SVN.fit() function
# TODO: Figure out the bug.

# We have our documents ready. Let's get labels ready now
# Let's map positive to 1 and negative to 2

labels_to_array = {"positive": 1, "negative": 2}
labels = [labels_to_array[tweet[1]] for tweet in ppTrainingData if tweet[1] in labels_to_array]
y = np.array(labels)

# Now, let's build the SVM classifier (a binary classifier)

from sklearn.svm import SVC
SVMClassifier = SVC()
SVMClassifier.fit(swn_X, y)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [28]:
#2d: Run classifier on the 100 downloaded tweets, and classify them as positive or negative.

# i) Naive Baye's:

NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in ppTestData]

In [ ]:
# ii) SVM:

SVMResultLabels = []
for tweet in ppTestData:
    tweet_sentence = ' '.join(tweet[0])
    svmFeatures = np.multiply(vectorizer.transform([tweet_sentence]).toarray(), np.array(swn_weights))
    SVMResultLabels.append(SVMClassifier.predict(svmFeatures[0]))
    
    # predict() returns a numpy array, get the first element of the array
    # There is only one element in the array.

In [30]:
# Step 3: Get majority vote and print the sentiment

#i) Naive Baye's

if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print "NB Result Positive Sentiment " + str(100 * NBResultLabels.count('positive') / len(NBResultLabels)) + "%"
else:
    print "NB Result Negative Sentiment " + str(100 * NBResultLabels.count('negative') / len(NBResultLabels)) + "%"

NB Result Negative Sentiment 62%


In [31]:
# ii) SVM

if SVMResultLabels.count(1) > SVMResultLabels.count(2):
    print "SVM Result Positive Sentiment " + str(100 * SVMResultLabels.count(1) / len(SVMResultLabels)) + "%"
else:
    print "SVM Result Negative Sentiment " + str(100 * SVMResultLabels.count(2) / len(SVMResultLabels)) + "%"

SVM Result Negative Sentiment 100%
